In [1]:
import tensorflow as tf
import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl

In [2]:
data_path = 'C:/Users/smartgrid_AI/Documents/kms/Python_Study/3. Panama.csv'
dat_source = pd.read_csv(data_path)
dat_source = pd.DataFrame(dat_source)
#dat_s = np.array(dat_source, dtype='float')

In [3]:
dat_source

,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school
0,03-01-2015 01:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0
1,03-01-2015 02:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0
2,03-01-2015 03:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0
3,03-01-2015 04:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0
4,03-01-2015 05:00,893.6865,25.973840,0.018981,0.017281,21.186089,23.075403,0.017059,0.001729,9.733589,22.481653,0.016456,0.064362,5.611724,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43770,31-12-2019 19:00,1301.6065,26.635645,0.018421,0.013165,13.184052,25.135645,0.018048,0.064240,3.086798,23.620020,0.016697,0.073425,3.865351,22,1,0
43771,31-12-2019 20:00,1250.9634,26.495935,0.018162,0.014713,13.443892,24.769373,0.017781,0.058838,3.659980,23.284998,0.016606,0.064362,4.171572,22,1,0
43772,31-12-2019 21:00,1193.6802,26.354456,0.017980,0.013836,13.442195,24.479456,0.017606,0.038086,3.769294,23.041956,0.016492,0.054260,4.045283,22,1,0
43773,31-12-2019 22:00,1130.4575,26.166895,0.017965,0.018486,13.420656,24.112207,0.017393,0.020386,3.872397,22.862207,0.016401,0.055557,3.843736,22,1,0


In [4]:
df = dat_source.iloc[:,1:-3]

In [5]:
df

,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav
0,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148
1,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471
2,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184
3,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621
4,893.6865,25.973840,0.018981,0.017281,21.186089,23.075403,0.017059,0.001729,9.733589,22.481653,0.016456,0.064362,5.611724
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43770,1301.6065,26.635645,0.018421,0.013165,13.184052,25.135645,0.018048,0.064240,3.086798,23.620020,0.016697,0.073425,3.865351
43771,1250.9634,26.495935,0.018162,0.014713,13.443892,24.769373,0.017781,0.058838,3.659980,23.284998,0.016606,0.064362,4.171572
43772,1193.6802,26.354456,0.017980,0.013836,13.442195,24.479456,0.017606,0.038086,3.769294,23.041956,0.016492,0.054260,4.045283
43773,1130.4575,26.166895,0.017965,0.018486,13.420656,24.112207,0.017393,0.020386,3.872397,22.862207,0.016401,0.055557,3.843736


In [6]:
min_max_scaler = preprocessing.StandardScaler()
norm_df = pd.DataFrame(min_max_scaler.fit_transform(df), 
                             columns=df.columns, 
                             index=df.index)

In [7]:
norm_df

,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav
0,-1.104627,-0.917472,0.128975,-1.005302,1.235019,-1.144115,-0.343719,-1.248909,0.837155,-0.849516,-0.223119,-0.593493,1.053902
1,-1.405228,-0.896895,0.177423,-1.001593,1.279262,-1.172214,-0.347505,-1.255035,0.924098,-0.884558,-0.256830,-0.689274,1.175793
2,-1.466758,-0.873880,0.249594,-1.015848,1.319529,-1.191036,-0.376265,-1.253863,0.971791,-0.904739,-0.276237,-0.790241,1.350571
3,-1.520063,-0.861615,0.326570,-1.003795,1.271366,-1.226412,-0.421498,-1.240290,0.883928,-0.912658,-0.271284,-0.907460,1.357848
4,-1.500774,-0.851751,0.383954,-0.988498,1.142105,-1.281600,-0.458811,-1.250375,0.690341,-0.925538,-0.291064,-0.953103,1.198760
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43770,0.607224,-0.451182,0.031316,-1.051020,0.023153,-0.585721,0.075035,-0.525595,-0.948741,-0.446036,-0.136983,-0.850407,0.176949
43771,0.345517,-0.535744,-0.131908,-1.027494,0.059487,-0.709435,-0.068952,-0.588224,-0.807395,-0.587154,-0.195263,-0.953103,0.356120
43772,0.049496,-0.621377,-0.246890,-1.040821,0.059250,-0.807359,-0.163363,-0.828833,-0.780439,-0.689528,-0.267975,-1.067556,0.282228
43773,-0.277218,-0.734902,-0.256234,-0.970188,0.056238,-0.931403,-0.278448,-1.034058,-0.755014,-0.765241,-0.326168,-1.052860,0.164302


In [8]:
import seaborn as sns
from keijzer import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from tensorflow.keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
from tensorflow.keras.utils import plot_model
from tensorflow.keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

Using TensorFlow backend.


In [9]:
norm_df = np.array(norm_df)

def create_dataset(dataset, timesteps, output_timesteps):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps):(i+timesteps+output_timesteps), :])
    return np.array(dataX), np.array(dataY)

timesteps = 120
output_timesteps = 24
num_features = 13
X, Y = create_dataset(norm_df, timesteps, output_timesteps)

In [10]:
X.shape, Y.shape

((43630, 120, 13), (43630, 24, 13))

In [11]:
trX = X[:31210, :, :]
vaX = X[31210:, :, :]
trY = Y[:31210, :, 0]
vaY = Y[31210:, :, 0]

In [12]:
trX.shape, vaX.shape, trY.shape, vaY.shape

((31210, 120, 13), (12420, 120, 13), (31210, 24), (12420, 24))

In [13]:
with tf.device('/gpu:0'):
    alpha=1.0
    gamma=1.2
    beta = 1.1785
    num_features = 13
    hfilters = 256
    hkernel_size1 = 2
    hkernel_size2 = 4
    visible1 = Input(shape=(timesteps, num_features))
    
    per1 = Permute((2,1))(visible1)
    den1a = Dense(timesteps, activation='tanh')(per1)
    den1b = Dense(timesteps, activation='sigmoid')(per1)
    den1 = Multiply()([den1a, den1b])
    per2 = Permute((2,1), name='attention_vec')(den1)
    mul1 = Multiply()([visible1, per2])
    
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=1)(mul1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    res01a = Add()([visible1, d1])   # (100, 25) (100, 25)
    res01b = Subtract()([visible1, d1])

    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01a)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    
    d1 = Multiply()([d1a, d1b])
    res02a = Add()([res01a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01b) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res02b = Subtract()([res01b, d2])   # (100, 25) (100, 25) 
    res02 = Concatenate()([res02a, res02b, res01a, res01b])
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=8)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    res03a = Add()([res02a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=8)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res03b = Subtract()([res02b, d2])   # (100, 25) (100, 25)
    res03 = Concatenate()([res03a, res03b, res02])
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=8)(res03)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    res04a = Add()([res03a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=8)(res03) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res04b = Subtract()([res02b, d2])   # (100, 25) (100, 25)
    res04 = Concatenate()([res04a, res04b, res03])
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res04)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    res05a = Add()([res04a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res04) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res05b = Subtract()([res04b, d2])   # (100, 25) (100, 25)
    res05 = Concatenate()([res05a, res05b, res04])
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res05)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=1)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])

    res06a = Add()([res05a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res05) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=1)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])

    res06b = Subtract()([res05b, d2])   # (100, 25) (100, 25)
    res06 = Concatenate()([res06a, res06b])
    
    res10 = Concatenate()([res02, res03, res04, res05, res06, visible1])   # 
    
    print('res10 :', res10.shape)  # (None, 24, 11) 
    
    out = Conv1D(720, 1, padding='same', activation='relu')(res10)   # 256, 11X10=110
    out = Dropout(0.2)(out)   #SpatialDropout1D
    
    out = Conv1D(240, 1, padding='same', activation='relu')(out) # 512,  110X5=550
    out = Dropout(0.2)(out)
    
    out = GlobalAveragePooling1D()(out) # pool_size=2, strides=1
    
    out = Dense(24)(out) 
    model = Model(inputs=[visible1], outputs=[out])
    
    print(model.summary())
    
    model.compile(loss='mae', optimizer='adam', metrics=['mse', 'mae'])
    early_stopping =EarlyStopping(monitor='val_loss', patience=150)
    batch_size = 100
    epochs = 1000

    history = LossHistory()
    history.init()
    
    #hist = model.fit(trX, trY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history, early_stopping])  # , checkpoint

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
res10 : (?, 120, 403)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 13)]    0                                            
__________________________________________________________________________________________________
permute (Permute)               (None, 13, 120)      0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 13, 120)      14520       permute[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 13, 120)      14520       permute[0][0]            

In [14]:
    %%time
    hist = model.fit(trX, trY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history, early_stopping])  # , checkpoint

Train on 31210 samples, validate on 12420 samples
Epoch 1/1000
31210/31210 [==============================] - 20s 632us/sample - loss: 0.5431 - mean_squared_error: 0.5167 - mean_absolute_error: 0.5431 - val_loss: 0.4446 - val_mean_squared_error: 0.3600 - val_mean_absolute_error: 0.4446 - loss: 0.8943 - mean_squared_error: 1.1562 - mean_absolut - ETA: 22s - loss: 0.8076 - mean_squa
Epoch 2/1000
31210/31210 [==============================] - 16s 502us/sample - loss: 0.4304 - mean_squared_error: 0.3510 - mean_absolute_error: 0.4304 - val_loss: 0.4203 - val_mean_squared_error: 0.3382 - val_mean_absolute_error: 0.4203
Epoch 3/1000
31210/31210 [==============================] - 16s 503us/sample - loss: 0.4086 - mean_squared_error: 0.3257 - mean_absolute_error: 0.4086 - val_loss: 0.4234 - val_mean_squared_error: 0.3449 - val_mean_absolute_error: 0.4234
Epoch 4/1000
31210/31210 [==============================] - 16s 505us/sample - loss: 0.3897 - mean_squared_error: 0.2988 - mean_absolute_error

In [15]:
trainPredict = model.predict(trX, batch_size=batch_size)
validPredict = model.predict(vaX, batch_size=batch_size)

In [16]:
trPredict = trainPredict.reshape([-1])
trainY = trY.reshape([-1])
valPredict = validPredict.reshape([-1])
validY = vaY.reshape([-1])

print('Error Training Score > MSE == ', (np.mean(np.square(trainY-trPredict))), ' MAE == ', mean_absolute_error(trainY, trPredict))
print('Error Validation Score > MSE == ', (np.mean(np.square(validY-valPredict))), ' MAE == ', mean_absolute_error(validY, valPredict))

Error Training Score > MSE ==  0.05285718749382127  MAE ==  0.18010500578783262
Error Validation Score > MSE ==  0.23965152241528467  MAE ==  0.3367353684909806


In [17]:
teX = X[-1,:,:].reshape(-1,120,13)
teX.shape

(1, 120, 13)

In [18]:
testPredict = model.predict(teX)
testPredict = testPredict.reshape(24,1)

In [19]:
trash = norm_df[-24:, 1:]
trash.shape

(24, 12)

In [20]:
YY = np.concatenate([testPredict, trash], axis=1)
YY.shape

(24, 13)

In [22]:
Or=min_max_scaler.inverse_transform(YY)

In [23]:
testval = Or[:,0]

In [24]:
testval

array([1204.62053199, 1146.90532943, 1095.14701709, 1060.1261208 ,
       1032.36619281, 1004.66263214,  998.79329175, 1016.37066672,
       1005.77483412, 1096.80285743, 1237.04640952, 1307.90922486,
       1369.47986469, 1398.68184783, 1389.63403449, 1401.17292098,
       1398.06301845, 1373.22127262, 1320.13875226, 1235.47732317,
       1257.74612657, 1299.50744992, 1271.96801998, 1240.27383334])

In [26]:
file = open("3. test_Panama.txt", "r")
realy = file.readlines()
file.close()

In [28]:
realy

['1032.263\n',
 '1017.5806\n',
 '1000.2797\n',
 '987.2383\n',
 '970.3667\n',
 '957.6767\n',
 '922.3729\n',
 '924.4692\n',
 '954.3947\n',
 '986.0648\n',
 '989.1948\n',
 '1005.6986\n',
 '1006.6785\n',
 '999.7711\n',
 '1000.8847\n',
 '1002.8859\n',
 '993.8271\n',
 '988.1441\n',
 '1086.8084\n',
 '1126.1699\n',
 '1130.5479\n',
 '1123.8589\n',
 '1072.263\n',
 '1076.9789']

In [31]:
with open("3. test_Panama.txt", "r") as file:
    realy2 = file.readlines()

In [32]:
realy2

['1032.263\n',
 '1017.5806\n',
 '1000.2797\n',
 '987.2383\n',
 '970.3667\n',
 '957.6767\n',
 '922.3729\n',
 '924.4692\n',
 '954.3947\n',
 '986.0648\n',
 '989.1948\n',
 '1005.6986\n',
 '1006.6785\n',
 '999.7711\n',
 '1000.8847\n',
 '1002.8859\n',
 '993.8271\n',
 '988.1441\n',
 '1086.8084\n',
 '1126.1699\n',
 '1130.5479\n',
 '1123.8589\n',
 '1072.263\n',
 '1076.9789']

In [50]:
file = open("3. test_Panama.txt", "r")
realy4 = np.array(1, dtype=np.float32)
while True:
    realy3 = file.readline()
    if not realy3:
        break
    realy4 = np.append(realy4,float(realy3.strip()))

file.close()

In [52]:
realy4 = realy4[1:]
realy4

array([1032.263 , 1017.5806, 1000.2797,  987.2383,  970.3667,  957.6767,
        922.3729,  924.4692,  954.3947,  986.0648,  989.1948, 1005.6986,
       1006.6785,  999.7711, 1000.8847, 1002.8859,  993.8271,  988.1441,
       1086.8084, 1126.1699, 1130.5479, 1123.8589, 1072.263 , 1076.9789])

In [53]:
print('Error Test Score > MSE == ', (np.mean(np.square(realy4-testval))), ' MAE == ', mean_absolute_error(realy4, testval))

Error Test Score > MSE ==  55871.714633049814  MAE ==  200.22796554125748


In [55]:
YYr = np.concatenate([realy4.reshape(-1,1), trash], axis=1)
YYr.shape

(24, 13)

In [58]:
YYr2 = min_max_scaler.fit_transform(YYr)

In [60]:
testy = YYr2[:,0]

In [61]:
testy = testy

array([ 0.30109646,  0.04720492, -0.25196633, -0.47748131, -0.769229  ,
       -0.98866749, -1.59914915, -1.56289943, -1.04542061, -0.49777374,
       -0.44364904, -0.15826143, -0.14131677, -0.26076116, -0.24150453,
       -0.2068993 , -0.36354622, -0.461818  ,  1.24430842,  1.92495679,
        2.0006622 ,  1.88499443,  0.99278592,  1.07433438])

In [64]:
tePredict = testPredict.reshape(-1)

In [65]:
print('Error Test Score > MSE == ', (np.mean(np.square(testy-tePredict))), ' MAE == ', mean_absolute_error(testy, tePredict))

Error Test Score > MSE ==  0.8854171240232519  MAE ==  0.7644576295528296
